In [1]:
import threading
from builtins import property

import numpy as np
from kompas3d import kAPI7 as KAPI7
from win32com.client import Dispatch, gencache, VARIANT
import pythoncom
from sqlalchemy import create_engine
import pandas as pd
import re



class KS3DAPP(object):
     def __init__(self):
        self.module7 = gencache.EnsureModule("{69AC2981-37C0-4379-84FD-5DD2F3C0A520}", 0, 1, 0)
        self.app7 = Dispatch('KOMPAS.Application.7')
        self.api7 = self.module7.IKompasAPIObject(
            self.app7._oleobj_.QueryInterface(self.module7.IKompasAPIObject.CLSID, pythoncom.IID_IDispatch))
        self.const = gencache.EnsureModule("{2CAF168C-7961-4B90-9DA2-701419BEEFE3}", 0, 1, 0).constants
        self.const6 = gencache.EnsureModule("{75C9F5D0-B5B8-4526-8681-9903C567D2ED}", 0, 1, 0).constants
        self.propMng = self.module7.IPropertyMng(
            self.app7._oleobj_.QueryInterface(self.module7.IPropertyMng.CLSID, pythoncom.IID_IDispatch))
        self.module5 = gencache.EnsureModule("{0422828C-F174-495E-AC5D-D31014DBBE87}", 0, 1, 0)
        self.api5 = self.module5.KompasObject(
            Dispatch("Kompas.Application.5")._oleobj_.QueryInterface(self.module5.KompasObject.CLSID,
                                                                     pythoncom.IID_IDispatch))
        self.app = self.api7.Application

class KS3Docs:
    def __init__(self, ks3dapp):
        self.ks3dapp = ks3dapp
        self.app = ks3dapp.app
        self.df = self.get_df()
        self.docs = {}
        self.docs_name = {}
        self.get_docs()

    def get_docs(self):
        for d in self.app.Documents:
            if self.docs.get(d.Reference) is None:
                doc = KS3Doc(self.ks3dapp, d)
                self.docs[d.Reference] = doc
                self.docs_name[d.Name] = doc


    def create_doc(self, d):
        doc = KS3Doc(self.ks3dapp, d)
        return doc


    def get_df(self):
        ls = []
        for d in self.app.Documents:
            ls.append([d.Name, d.Active, d.Path, d.PathName, d.Reference])
        df = pd.DataFrame(ls, columns=['name', 'active', 'path', 'path_name', 'ref'])
        return df

    @property
    def df(self):
        return self.get_df()

    @df.setter
    def df(self, value):
        self._df = value

    def update_df(self):
        self.df = self.get_df()

    def get_doc_by_path(self, path_name):
        return self.app.Documents.Item(path_name)

    def get_doc_by_ref(self, ref):
        return self.app.Documents.Item(ref)

    def get_doc_by_index(self, index):
        return self.app.Documents.Item(index)

class KS3Doc:

    def __init__(self, ks3dapp, idoc):
        self.ks3dapp = ks3dapp

        self.doc = idoc
        self.doc1 = self.ks3dapp.module7.IKompasDocument1(self.doc)
        self.doc2d = self.ks3dapp.module7.IKompasDocument2D(self.doc)
        self.doc2d1 = self.ks3dapp.module7.IKompasDocument2D1(self.doc2d)
        self.doc2D5 = self.ks3dapp.api5.ActiveDocument2D()

        self.views_and_layers_manager = self.doc2d.ViewsAndLayersManager
        self.views = self.views_and_layers_manager.Views
        self.view = self.views.ActiveView

        self.drawing_container = self.ks3dapp.module7.IDrawingContainer(self.view)
        self.symbols_2d_container = self.ks3dapp.module7.ISymbols2DContainer(self.view)
        self.drawing_texts = self.drawing_container.DrawingTexts

        self.df_texts = None
        self.ds_texts = None
        self.df_tables = None
        self.ds_tables = None
        self.df_lines = None
        self.ds_lines = None
        self.df_polylines = None
        self.ds_polylines = None
        self.df_circles = None
        self.ds_circles = None

        self._init_df_ds()

    def _init_df_ds(self):
        self._init_df_ds_texts()
        self._init_df_ds_circles()
        self._init_df_ds_lines()
        # self._init_df_ds_polylines()


    def _init_df_ds_texts(self):
        ls = []
        ds = {}
        for i in range(self.drawing_texts.Count):
            t = self.drawing_texts.DrawingText(i)
            t1 = self.ks3dapp.module7.IText(t)
            ls.append([t1.Str, t.X + t.Width / 2, t.Y + t.Height / 2, t.Reference])
            ds[t.Reference] = t
        self.df_texts = pd.DataFrame(ls, columns=['text', 'x', 'y', 'ref'])
        self.ds_texts = ds

    def _init_df_ds_lines(self):
        ls = []
        ds = {}
        for i in range(self.drawing_container.LineSegments.Count):
            t = self.drawing_container.LineSegments.LineSegment(i)
            ls.append([t.X1, t.Y1, t.X2, t.Y2, t.Reference])
            ds[t.Reference] = t
        self.df_lines = pd.DataFrame(ls, columns=['x1', 'y1', 'x2', 'y2', 'ref'])
        self.ds_lines = ds

    def _init_df_ds_polylines(self):
        ls = []
        ds = {}
        for i in range(self.drawing_container.PolyLines2D.Count):
            t = self.drawing_container.PolyLines2D.PolyLine2D(i)
            ls.append([t.X1, t.Y1, t.X2, t.Y2, t.Reference])
            ds[t.Reference] = t
        self.df_polylines = pd.DataFrame(ls, columns=['x1', 'y1', 'x2', 'y2', 'ref'])
        self.ds_polylines = ds

    def _init_df_ds_circles(self):
        ls = []
        ds = {}
        for i in range(self.drawing_container.Circles.Count):
            t = self.drawing_container.Circles.Circle(i)
            ls.append([t.Xc, t.Yc, t.Radius, t.Reference])
            ds[t.Reference] = t
        self.df_circles = pd.DataFrame(ls, columns=['x', 'y', 'radius', 'ref'])
        self.ds_circles = ds

class TextTools:
    def __init__(self, ks, doc):
        self.KS = ks
        self.doc = doc
        self.ds = self.doc.ds_texts
        self.df = self.doc.df_texts
        self.df_circles = self.doc.df_circles
        self.df_lines = self.doc.df_lines

    def get_idraw_text(self, ref):
        return self.ds[ref]

    def get_itext(self, ref):
        idraw_text = self.get_idraw_text(ref)
        return self.KS.module7.IText(idraw_text)

    def select_ref(self, ref):
        idraw_text = self.get_idraw_text(ref)
        itext = self.KS.module7.IText(idraw_text)
        self.doc.doc2d1.SelectionManager.Select(itext)

    def select_refs(self, refs):
        for ref in refs:
            self.select_ref(ref)

    def df_match(self, match):
        return self.df[self.df['text'].str.match(match)]

    def find_text(self, x, y,  dx_left, dx_right, dy_up, dy_down, df_poisk=None, return_only_df=False, col_name_text='text',  lst_filter=None, lst_exclude=None):
        if df_poisk is None:
            df_poisk = self.df

        df = df_poisk[(df_poisk['x'] >= x - dx_left) & (df_poisk['x'] <= x + dx_right) & (df_poisk['y'] >= y - dy_down) & (df_poisk['y'] <= y + dy_up)]

        if lst_filter is not None:
            # выбрать только объекты из фильтра
            df = df[df['text'].isin(lst_filter)]

        if lst_exclude is not None:
            # выбрать только объекты из фильтра
            df = df[~df['text'].isin(lst_exclude)]

        if len(df) == 0:
            return None
        else:
            ref1 = int(df.iloc[0]['ref'])
            if return_only_df:
                return df
            else:
                res = {
                    'text': df.iloc[0][col_name_text],
                    'ref': ref1,
                    'x': df.iloc[0]['x'],
                    'y': df.iloc[0]['y'],
                    'width': self.ds[ref1].Width,
                    'df': df
                }
            return res

    # найти объект в пределах отклонения по X Y, и содержащий указанный текст
    def find_text_ref(self, ref,  dx_left, dx_right, dy_up, dy_down, df_poisk=None, return_only_df=False, col_name_text='text', lst_filter=None, lst_exclude=None):
        x, y = self.get_xy(ref)

        dt = self.find_text(x, y, dx_left, dx_right, dy_up, dy_down, df_poisk, return_only_df, col_name_text, lst_filter, lst_exclude)
        if dt is not None:
            # исключить текущий объект
            df = dt['df']
            df = df[df['ref'] != ref]
            dt['df'] = df

            if len(df) == 0:
                return None
            else:
                ref1 = int(df.iloc[0]['ref'])
                if return_only_df:
                    return df
                else:
                    res = {
                        'text': df.iloc[0][col_name_text],
                        'ref': ref1,
                        'x': df.iloc[0]['x'],
                        'y': df.iloc[0]['y'],
                        'width': self.ds[ref1].Width,
                        'df': df
                    }
                return res
        else:
            return None

    def find_line(self, x, y,  dx_left, dx_right, dy_up, dy_down):
        df = self.df_lines[(self.df_lines['x1'] >= x - dx_left) & (self.df_lines['x1'] <= x + dx_right) & (self.df_lines['y1'] >= y - dy_down) & (self.df_lines['y1'] <= y + dy_up)]

        df1 = self.df_lines[(self.df_lines['x2'] >= x - dx_left) & (self.df_lines['x2'] <= x + dx_right) & (self.df_lines['y2'] >= y - dy_down) & (self.df_lines['y2'] <= y + dy_up)]

        if len(df) != 0:
            ref1 = int(df.iloc[0]['ref'])
            res = {
                'ref': ref1,
                'x': df.iloc[0]['x1'],
                'y': df.iloc[0]['y1'],
                'df': df
            }
            return res

        if len(df1) != 0:
            ref1 = int(df1.iloc[0]['ref'])
            res = {
                'ref': ref1,
                'x': df1.iloc[0]['x2'],
                'y': df1.iloc[0]['y2'],
                'df': df1
            }
            return res


    def find_cirle(self, x, y,  dx_left, dx_right, dy_up, dy_down, radius_lt=None, radius_gt=None,  df_poisk=None):
        if df_poisk is None:
            df_poisk = self.df_circles
        df = df_poisk[(df_poisk['x'] >= x - dx_left) & (df_poisk['x'] <= x + dx_right) & (df_poisk['y'] >= y - dy_down) & (df_poisk['y'] <= y + dy_up)]
        # выбрать только объекты с указанным радиусом
        if radius_lt is not None:
            df = df[df['radius'] < radius_lt]
        if radius_gt is not None:
            df = df[df['radius'] > radius_gt]

        if len(df) == 0:
            return None
        else:
            ref1 = int(df.iloc[0]['ref'])
            res = {
                'ref': ref1,
                'x': df.iloc[0]['x'],
                'y': df.iloc[0]['y'],
                'radius': df.iloc[0]['radius'],
                'df': df
            }
            return res

    def find_cirle_ref(self, ref,  dx_left, dx_right, dy_up, dy_down, radius_lt=None, radius_gt=None,  df_poisk=None):
        x, y = self.get_xy(ref)
        dt = self.find_cirle(x, y, dx_left, dx_right, dy_up, dy_down, radius_lt, radius_gt,  df_poisk)
        return dt

    def get_xy(self, ref):
        df = self.df[self.df['ref']==ref]
        x = df.iloc[0]['x']
        y = df.iloc[0]['y']
        return x, y

    def add_text(self, text, x, y, ref_text_font, rotation=0):

        draw_txt = self.doc.drawing_texts.Add()
        draw_txt.X = x
        draw_txt.Y = y

        itext = self.KS.module7.IText(draw_txt)
        itext.Str = text

        ds_font = self.font_copy(ref_text_font)
        self.set_font(itext, **ds_font)
        draw_txt.Update()

        draw_txt.X -= draw_txt.Width / 2
        draw_txt.Y -= draw_txt.Height / 2
        if rotation != 0:
            draw_txt.Angle = rotation
        draw_txt.Update()

    def replace(self, ref, text):
        if ref is not None:
            idraw_text = self.ds[ref]
            if idraw_text is not None:
                itext = self.KS.module7.IText(idraw_text)
                itext.Str = text
                idraw_text.Update()


    def font_copy(self, ref_text_font):
        idraw_text_font = self.ds[ref_text_font]
        itext_font = self.KS.module7.IText(idraw_text_font)

        tf = self.KS.module7.ITextFont(itext_font.TextLines[0].TextItems[0])

        ds_font = {
            'height': tf.Height,
            'name': tf.FontName,
            'bold': tf.Bold,
            'italic': tf.Italic,
            'underline': tf.Underline,
            'widthfactor': tf.WidthFactor,
            'color': tf.Color
        }

        # self.set_font(itext, tf.Height, tf.FontName, tf.Bold, tf.Italic, tf.Underline, tf.WidthFactor, tf.Color)
        return ds_font

    def set_font(self, itext, height=3, name='Arial', bold=False, italic=False, underline=False, widthfactor=1.0, color=0):
        self.set_font_lines(itext, None, height, name, bold, italic, underline, widthfactor, color)

    def set_font_lines(self, itext, lines=None, height=3, name='Arial', bold=False, italic=False, underline=False, widthfactor=1.0, color=0):
        if lines is None:
            lines = itext.TextLines
        for line in lines:
            self.set_font_items(line.TextItems, height, name, bold, italic, underline, widthfactor, color)


    def set_font_items(self, items, height=3, name='Arial', bold=False, italic=False, underline=False, widthfactor=1.0, color=0):
        for item in items:
            self.set_font_item(item, height, name, bold, italic, underline, widthfactor, color)

    def set_font_item(self, item, height=3, name='Arial', bold=False, italic=False, underline=False, widthfactor=1.0, color=0):
        tf = self.KS.module7.ITextFont(item)
        tf.Height = height
        tf.FontName = name
        tf.Bold = bold
        tf.Italic = italic
        tf.Underline = underline
        tf.WidthFactor = widthfactor
        tf.Color = color
        item.Update()

df8 = None
df9 = None
i9 = None

class Tools:
    def __init__(self, text_tools_st, text_tools_rcn):
        self.df = None
        self.text_tools_st = text_tools_st
        self.text_tools_rcn = text_tools_rcn

        self.ls_contact = ['1', '2', '3', '4', '11', '12', '13', '21', '22', '23', '31', '32', '33', '41', '42', '43',
                           '51', '52', '53', '61', '62', '63', '71', '72', '73', '81', '82', '83', '111', '112', '113',
                           '121', '122', '123']

        self.ds_st_b = {
            'df_101': ['В19', 'В110', 'В111', 'В112', 'В29', 'В210', 'В211', 'В212', 'В18', 'В28', ],
            'df_C': ['В16', 'В17', 'В26', 'В27', 'В25'],
            'df_contacts': ['В18', 'В28'],
            'df_bok': ['Б9', 'Б10', 'Б11', 'Б12', 'Б4', 'Б3', 'Б2', 'Б1'],
        }
        self.df_st_b = None

        self.df_st_shina = None
        self.df_st_sh_p_ls = None
        self.df_B = None
        self.df_101 = None
        self.df_C = None

        self.df_sx_sh_circle = None
        self.df_sx_contacts = None
        self.df_st_shina_montag = pd.DataFrame(columns=['st', 'sh', 'loc', 'contact', 'polka'])
        self.ls_st_name = ['1', '2', '3']
        self.ls_st_sh_name = ['М', 'П', 'МСХ', 'СХМ']

    def sx_search_contacts(self):
        global df9, i9
        ls = []
        df1 = self.text_tools_rcn.df_match('^[1-3]-\d{2,3}')

        for i in range(len(df1)):
            i9 = i
            ref1 = df1.iloc[i]['ref']
            t_loc = df1.iloc[i]['text']
            dt_name = self.text_tools_rcn.find_text_ref(ref1, 8, 8, 0, 5, lst_exclude=self.ls_contact)
            if dt_name is not None:
                t_name = dt_name['text']
                dc_name = self.text_tools_rcn.find_cirle_ref(dt_name['ref'], 20, 20, 8, 9, radius_gt=3)
                if dc_name is not None:
                    dt_contact = self.text_tools_rcn.find_text(dc_name['x'], dc_name['y'], 10, 10, 10, 10,
                                                               lst_filter=self.ls_contact)
                else:
                    dt_contact = self.text_tools_rcn.find_text_ref(dt_name['ref'], 20, 20, 0, 12,
                                                                   lst_filter=self.ls_contact)

                if dt_contact is not None:
                    df9 = dt_contact
                    df_contact = dt_contact['df']
                    if df_contact['text'].tolist() == ['1', '2', '3', '4']:
                        df_contact = df_contact[df_contact['text'].isin(['2', '4'])]
                    ds = self._direction_contact(df_contact)
                    group = t_loc + ';' + ':'.join(df_contact['text'].values.tolist())

                    for j in range(len(df_contact)):
                        t_contact = df_contact.iloc[j]['text']
                        t_ref = df_contact.iloc[j]['ref']
                        t_x = df_contact.iloc[j]['x']
                        t_y = df_contact.iloc[j]['y']
                        st, p, r = self.get_loc(t_loc)
                        ls.append([t_loc, st, p, r, t_name, t_contact, t_ref, t_x, t_y, group, ds[t_contact][0],
                                   ds[t_contact][1]])

        self.df_sx_contacts = pd.DataFrame(ls, columns=['loc', 'st', 'p', 'r', 'name', 'contact', 'ref', 'x', 'y', 'group', 'direction',
                                        'side'])

        self.df_sx_contacts[['b1', 'b2', 'l1', 'l2', 'pit']] = None

    def sx_search_v(self):
        self.df_B = self.text_tools_rcn.df_match('^[1-3]-[ВБ]')
        self.df_B = self.df_B.apply(self._rep_xy_on_circle, axis=1)
        self.df_101 = self.text_tools_rcn.df_match('^10[1-5]-')
        self.df_101 = self.df_101.apply(self._rep_xy_on_circle, axis=1)
        self.df_C = self.text_tools_rcn.df_match('^C[1-8]:')
        self.df_C = self.df_C.apply(self._rep_xy_on_circle, axis=1)

        # отсортировать по x возрастанию потом y по убыванию
        self.df_B = self.df_B.sort_values(by=['x', 'y'], key=lambda x: x.map(dict(zip(x, range(len(x))))))

    def sx_search_sh_circle(self):
        df_sh = self.text_tools_rcn.df_match(r'[1-3]-Ш')

        for i in range(len(df_sh)):
            ref1 = df_sh.iloc[i]['ref']
            ds = self.text_tools_rcn.find_text_ref(ref1, 6, 6, 0, 5)
            if ds is not None:
                df2 = None
                name_sh = ds['text']  # 'П'
                ds1 = self.text_tools_rcn.find_cirle_ref(ds['ref'], 5, 5, 0, 10, radius_lt=2)
                if ds1 is not None:
                    df2 = ds1['df']
                    df2['pit'] = name_sh

                ds_lines = self.text_tools_rcn.find_line(ds['x'], ds['y'], 5, 5, 0, 10)

                if ds_lines is not None:
                    df_lines = ds_lines['df']

                    is_line = False
                    for i in range(len(df_lines)):
                        res = df_lines.iloc[i]
                        if res['x1'] == res['x2']:
                            is_line = True
                            break
                    if is_line:
                        df2 = self.text_tools_rcn.find_cirle_ref(ds['ref'], 5, 5, 0, 180, radius_lt=2)['df']
                        df2['pit'] = name_sh

                if df2 is not None:
                    if self.df_sx_sh_circle is None:
                        self.df_sx_sh_circle = df2.copy()
                    else:
                        self.df_sx_sh_circle = self.df_sx_sh_circle.append(df2)

        self.sx_set_pit_contact()

    def sx_set_pit_contact(self):
        df1 = self.df_sx_contacts[self.df_sx_contacts['b1'].isnull()]
        for i in range(len(df1)):
            res = df1.iloc[i]
            ref1 = res['ref']
            df2 = self._find_link_by_sh(self.df_sx_sh_circle, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])
            if df2 is not None:
                df3 = df2['df']
                pit = df3['pit'].values[0]

                self.df_sx_contacts = self.set_val_df(self.df_sx_contacts, ref1, 'pit', pit)

    def st_init_shina(self):
        df5 = self.text_tools_st['1'].df_match(r'.Ш-[А-Я]')
        df5 = df5.reset_index(drop=True)
        df5['st'] = '1'
        df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
        df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
        self.df_st_shina = df5.copy()

        df5 = self.text_tools_st['2'].df_match(r'.Ш-[А-Я]')
        df5 = df5.reset_index(drop=True)
        df5['st'] = '2'
        df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
        df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
        self.df_st_shina = self.df_st_shina.append(df5)

        df5 = self.text_tools_st['3'].df_match(r'.Ш-[А-Я]')
        df5 = df5.reset_index(drop=True)
        df5['st'] = '3'
        df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
        df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
        self.df_st_shina = self.df_st_shina.append(df5)

        self.df_st_shina = self.df_st_shina.reset_index(drop=True)

        self.df_st_sh_p_ls = self.df_st_shina.groupby(['st', 'pit'])['p'].agg(lambda x: list(x)).reset_index()

    def st_init_v(self):
        ls_B = ['В15', 'В25', 'В16', 'В26', 'В17', 'В27', 'В18', 'В28', 'В19', 'В29', 'В110', 'В111', 'В112', 'В210',
                'В211', 'В212', 'Б9', 'Б10', 'Б11', 'Б12', 'Б4', 'Б3', 'Б2', 'Б1']
        ls_st = ['1', '2', '3']
        ls = []
        for st in ls_st:
            for b in ls_B:
                for i in range(1, 21):
                    ls.append([st, b, i])
        self.df_st_b = pd.DataFrame(ls, columns=['st', 'b', 'n'])
        self.df_st_b[['l1', 'l2', 'p']] = ''



    def st_montag_B(self):
        global df8, df9
        for i in range(len(self.df_B)):
            res = self.df_B.iloc[i]
            text_b = res['text']
            df1, df_name1 = self._find_link_by_b(res['x'], res['y'], 10, 0, 5, 5, side='left')
            df2, df_name2 = self._find_link_by_b(res['x'], res['y'], 0, 0, 5, 5, side='right')
            df8 = df1
            df9 = df2
            loc = None
            contact = None
            df_name_st = None
            df_name_no_st = None
            if df_name1 == 'df_contacts':
                loc = df1.iloc[0]['loc']
                df_cont = df1
                contact = df1.iloc[0]['contact']
                b1_dev = df1.iloc[0]['b1']
                b2_dev = df1.iloc[0]['b2']
                df_name_st = df_name1
            if df_name1 == 'df_101' or df_name1 == 'df_C':
                df_no_cont = df1
                b_st = df1.iloc[0]['text']
                df_name_no_st = df_name1

            if df_name2 == 'df_contacts':
                loc = df2.iloc[0]['loc']
                df_cont = df2
                contact = df2.iloc[0]['contact']
                b1_dev = df2.iloc[0]['b1']
                b2_dev = df2.iloc[0]['b2']
                df_name_st = df_name2
            if df_name2 == 'df_101' or df_name2 == 'df_C':
                df_no_cont = df2
                b_st = df2.iloc[0]['text']
                df_name_no_st = df_name2

            if loc is not None and df_name_no_st is not None:
                st, p, r = self.get_loc(loc)
                ls = self.ds_st_b[df_name_no_st]
                if 'Б' in text_b:
                    ls = self.ds_st_b['df_bok']
                df_t = self.df_st_b[
                    (self.df_st_b['st'] == st) & (self.df_st_b['b'].isin(ls)) & (self.df_st_b['l1'] == '')]
                df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
                df_bb = df_t.head(1)
                b, n = df_bb['b'].values[0], df_bb['n'].values[0]  #'B110', 1
                s_b = f'{st}-{b}-{n}'

                self.text_tools_rcn.replace(res['ref'], s_b)
                self.add_st_contact(loc=loc, contact=contact, loc_to=[f'{b}-{n}'])
                self.add_st_v(st, b, str(n), [f'{p}{r}-{contact}', b_st])

                # self.set_val_df(self.df_st_b, df_bb['ref'], 'l1', loc)
                # self.set_val_df(self.df_st_b, df_bb['ref'], 'l2', b_st)
                self.df_st_b.loc[df_bb.index, 'l1'] =  loc
                self.df_st_b.loc[df_bb.index, 'l2'] =  b_st

                if self.get_val_df_first(df_cont, 'b1') is None:
                    # self.set_val_df(self.df_sx_contacts, df_cont['ref'], 'b1', f'{b}-{n}')
                    self.df_sx_contacts.loc[df_cont.index, 'b1'] =  f'{b}-{n}'
                else:
                    # self.set_val_df(self.df_sx_contacts, df_cont['ref'], 'b2', f'{b}-{n}')
                    self.df_sx_contacts.loc[df_cont.index, 'b2'] =  f'{b}-{n}'

            if loc is None and df_name_no_st is not None:
                st = res['text'][0]
                ls = self.ds_st_b[df_name_no_st]
                if 'Б' in text_b:
                    ls = self.ds_st_b['df_bok']
                df_t = self.df_st_b[
                    (self.df_st_b['st'] == st) & (self.df_st_b['b'].isin(ls)) & (self.df_st_b['l1'] == '')]
                df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
                df_bb = df_t.head(1)
                b, n = df_bb['b'].values[0], df_bb['n'].values[0]  #'B110', 1
                s_b = f'{st}-{b}-{n}'

                self.text_tools_rcn.replace(res['ref'], s_b)
                self.add_st_v(st, b, str(n), ['', b_st])

                # self.set_val_df(self.df_st_b, df_bb['ref'], 'l1', 'резерв')
                # self.set_val_df(self.df_st_b, df_bb['ref'], 'l2', b_st)
                self.df_st_b.loc[df_bb.index, 'l1'] =  'резерв'
                self.df_st_b.loc[df_bb.index, 'l2'] =  b_st

    def st_montag_shina(self):
        for st_name in self.ls_st_name:
            for sh_name in self.ls_st_sh_name:
                df1 = self.filter_df(self.df_sx_contacts, ('st', st_name), ('pit', sh_name))
                if len(df1) > 0:
                    df1 = df1.sort_values(by=['p', 'r', 'contact'])
                    for i in range(len(df1)):
                        loc = df1['loc'].values[i]
                        contact = df1['contact'].values[i]
                        polka = str(self.get_nearest_sh(loc, sh_name))
                        cont_sh = self._st_get_contact_sh(st_name,  polka, sh_name, loc)

                        p = df1['p'].values[i]
                        r = df1['r'].values[i]
                        if p == polka:
                            loc_to = f'Ш-{cont_sh}'
                            loc_to_sh = f'{r}-{contact}'
                        else:
                            loc_to = f'{polka}Ш-{cont_sh}'
                            loc_to_sh = f'{p}{r}-{contact}'

                        self.add_st_contact(loc, contact, [loc_to, sh_name])


                        self.add_st_sh(st_name, f'{polka}Ш-{sh_name}', cont_sh, loc_to_sh)



    def get_nearest_sh(self, loc, pit):
        st, p, r = self.get_loc(loc)
        ls1 = self.df_st_sh_p_ls[(self.df_st_sh_p_ls['st'] == st) & (self.df_st_sh_p_ls['pit'] == pit)]['p']
        if len(ls1) > 0:
            ls1 = ls1.values[0]
            ls1 = [int(x) for x in ls1]
            return self._find_nearest(ls1, int(p))

    def add_st_contact(self, loc, contact, loc_to):

        st, p, r = self.get_loc(loc)

        text_tools = self.text_tools_st[st]

        df = text_tools.doc.df_texts

        df1 = df[df['text'] == f'Полка {p}']
        ref1 = df1.iloc[0]['ref']
        d1 = text_tools.find_text_ref(ref1, 360, 30, 270, -250)
        df1 = d1['df']

        df1 = df1[df1['text'] == r]
        ref1 = df1.iloc[0]['ref']
        d1 = text_tools.find_text_ref(ref1, 25, -15, 0, 240)
        df1 = d1['df']

        df1 = df1[df1['text'] == contact]
        ds = df1.iloc[0].to_dict()

        if len(loc_to) == 1:
            text_tools.add_text(loc_to[0], ds['x'] + 15, ds['y'], ref_text_font=ds['ref'])
        elif len(loc_to) == 2:
            text_tools.add_text(loc_to[0], ds['x'] + 15, ds['y'], ref_text_font=ds['ref'])
            text_tools.add_text(loc_to[1], ds['x'] + 35, ds['y'], ref_text_font=ds['ref'])
        elif len(loc_to) == 3:
            text_tools.add_text(loc_to[0], ds['x'] + 15, ds['y'] + 2, ref_text_font=ds['ref'])
            text_tools.add_text(loc_to[1], ds['x'] + 15, ds['y'] - 2, ref_text_font=ds['ref'])
            text_tools.add_text(loc_to[2], ds['x'] + 35, ds['y'], ref_text_font=ds['ref'])

        return True

    def add_st_v(self, st, v, contact, loc_to):

        text_tools = self.text_tools_st[st]

        df_v = text_tools.df_match(v)
        if len(df_v) == 0:
            return False

        ref_v = df_v.iloc[0]['ref']

        d1 = text_tools.find_text_ref(ref_v, 7, 7, 0, 85)
        df1 = d1['df']

        df1 = df1[df1['text'] == contact]
        ds = df1.iloc[0].to_dict()

        if int(contact) % 2 == 1:
            if len(loc_to) == 1:
                text_tools.add_text(loc_to[0], ds['x'] - 12, ds['y'], ref_text_font=ds['ref'])
            elif len(loc_to) == 2:
                text_tools.add_text(loc_to[0], ds['x'] - 12, ds['y'], ref_text_font=ds['ref'])
                text_tools.add_text(loc_to[1], ds['x'] - 12, ds['y'] - 4, ref_text_font=ds['ref'])
        else:
            if len(loc_to) == 1:
                text_tools.add_text(loc_to[0], ds['x'] + 12, ds['y'], ref_text_font=ds['ref'])
            elif len(loc_to) == 2:
                text_tools.add_text(loc_to[0], ds['x'] + 12, ds['y'], ref_text_font=ds['ref'])
                text_tools.add_text(loc_to[1], ds['x'] + 12, ds['y'] - 4, ref_text_font=ds['ref'])

    def add_st_sh(self, st, sh_st, contact, loc_to):

        text_tools = self.text_tools_st[st]

        df1 = text_tools.df_match(sh_st)
        ref1 = df1.iloc[0]['ref']
        d1 = text_tools.find_text_ref(ref1, 90, 110, -25, 40)
        df1 = d1['df']
        df1 = df1[df1['text'] == str(contact)]
        ds = df1.iloc[0].to_dict()

        text_tools.add_text(loc_to, ds['x'] + 1, ds['y'] + 8, ref_text_font=ds['ref'], rotation=90)

        return True

    # выбрать из списка ls1 число ближйшее к заданному
    def _find_nearest(self, ls, value):
        ls = np.array(ls)
        idx = (np.abs(ls - value)).argmin()
        return ls[idx]

    def _rep_xy_on_circle(self, row):
        res = self.text_tools_rcn.find_cirle(row['x'], row['y'], 10, 10, 0, 8, radius_lt=2)
        if res is not None:
            row['x'] = res['x']
            row['y'] = res['y']
        return row

    def _find_link_by_b(self, x, y, dx_left=1, dx_right=1, dy_top=1, dy_bottom=1, side='left'):
        df_name = 'df_contacts'
        res1 = None
        for i in range(30):
            res1 = self.text_tools_rcn.find_text(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=self.df_sx_contacts,
                                                 return_only_df=True)
            if res1 is not None:
                df_name = 'df_contacts'
                break

            res1 = self.text_tools_rcn.find_text(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=self.df_101,
                                                 return_only_df=True)
            if res1 is not None:
                df_name = 'df_101'
                break

            res1 = self.text_tools_rcn.find_text(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=self.df_C,
                                                 return_only_df=True)
            if res1 is not None:
                df_name = 'df_C'
                break

            if side == 'left':
                dx_left += 5
            else:
                dx_right += 5

        if res1 is not None:
            return res1, df_name
        return None, None

    def _find_link_by_sh(self, df_poisk, x, y, dx_left=1, dx_right=1, dy_top=1, dy_bottom=1, side='left'):
        res1 = None
        for i in range(30):
            res1 = self.text_tools_rcn.find_cirle(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=df_poisk)
            if res1 is not None:
                break

            if side == 'left':
                dx_left += 5
            else:
                dx_right += 5

        if res1 is not None:
            return res1
        return None

    def _direction_contact(self, df):
        if len(df) > 1:
            c1 = df.iloc[0]['text']
            c2 = df.iloc[1]['text']
            x1 = df.iloc[0]['x']
            x2 = df.iloc[1]['x']
            y1 = df.iloc[0]['y']
            y2 = df.iloc[1]['y']

            dx = abs(x1 - x2)
            dy = abs(y1 - y2)

            if dx > 2:
                if x1 < x2:
                    return {c1: ['horizontal', 'left'], c2: ['horizontal', 'right']}
                else:
                    return {c1: ['horizontal', 'right'], c2: ['horizontal', 'left']}
            if dy > 2:
                if y1 > y2:
                    return {c1: ['vertical', 'top'], c2: ['vertical', 'bottom']}
                else:
                    return {c1: ['vertical', 'bottom'], c2: ['vertical', 'top']}

    def _st_get_contact_sh(self, st,  polka, sh, loc):
        df1 = self.filter_df(self.df_st_shina_montag, ('st', st), ('sh', sh), ('polka', polka))
        if len(df1) == 0:
            self.df_st_shina_montag = self.df_st_shina_montag.append({'st': st, 'sh': sh, 'loc': loc, 'contact': '1', 'polka': polka}, ignore_index=True)
            return '1'
        else:
            cont = str(int(df1.tail(1)['contact'].values[0]) + 1)
            self.df_st_shina_montag = self.df_st_shina_montag.append({'st': st, 'sh': sh, 'loc': loc, 'contact': cont, 'polka': polka}, ignore_index=True)
            return cont

    def get_loc(self, loc):
        ls = loc.split('-')
        st = ls[0]
        pr = ls[1]
        if len(pr) == 2:
            p = pr[0]
            r = pr[1]
        else:
            p = pr[0:2]
            r = pr[2]
        return st, p, r

    def set_val_df(self, df, ref, col, value):
        if df is not None:
            ser = df[df['ref'] == ref][col]
            if len(ser) > 0:
                df.loc[df['ref'] == ref, col] = value
        return df

    def get_val_df(self, df, ref, col):
        if df is not None:
            df1 = df[df['ref'] == ref]
            if len(df1) > 0:
                return df1[col].values[0]

    def get_val_df_first(self, df, col):
        if df is not None:
            df1 = df.head(1)
            if len(df1) > 0:
                return df[col].values[0]

    def filter_df(self, df, col1_val1, col2_val2=None, col3_val3=None):
        col1, val1 = col1_val1
        df1 = df[df[col1] == val1]
        if col2_val2 is not None:
            col2, val2 = col2_val2
            df1 = df1[df1[col2] == val2]

        if col3_val3 is not None:
            col3, val3 = col3_val3
            df1 = df1[df1[col3] == val3]
        return df1



In [2]:
ks3dapp = KS3DAPP()

ks3docs = KS3Docs(ks3dapp)

# docs ={}
# for i in ks3docs.docs_name.keys():
#     s1 = re.search('л\.\d{1,2}\s+(.+)\.frw', i).group(1)
#     docs[s1] = i
# print(docs)

docs = {
    'rcn12': '12.frw',
    'rcn': '2517-АТР.2 л.3 Схемы РЦ Неч..frw',
    'rcn5': '2517-АТР.2 л.4 Схемы РЦ Чет..frw',
    'rcn1': '2517-АТР.2 л.9 Переезды.frw',
    'rcn2': '2517-АТР.2 л.11 Ограждение.frw',
    'rcn3': '2517-АТР.2 л.12 Загальні схеми.frw',
    'yv': '2517-АТР.2 л.10 Схемы увязок.frw',
    'c1': '2517-АТР.2 л.18 Статив 1.frw',
    'c2': '2517-АТР.2 л.19 Статив 2.frw',
    'c3': '2517-АТР.2 л.20 Статив 3.frw',
}
doc_rcn = ks3docs.docs_name[docs['rcn12']]
doc_st1 = ks3docs.docs_name[docs['c1']]
doc_st2 = ks3docs.docs_name[docs['c2']]
doc_st3 = ks3docs.docs_name[docs['c3']]

text_tools_rcn = TextTools(ks3dapp, doc_rcn)

text_tools_st = {
    '1': TextTools(ks3dapp, doc_st1),
    '2': TextTools(ks3dapp, doc_st2),
    '3': TextTools(ks3dapp, doc_st3),
}

ts = Tools(text_tools_st, text_tools_rcn)

ts.sx_search_contacts()
ts.st_init_v()

ts.sx_search_v()
ts.st_montag_B()

# Питания
ts.sx_search_sh_circle()
ts.st_init_shina()
ts.st_montag_shina()


In [4]:
ts = Tools(text_tools_st, text_tools_rcn)
ts.sx_search_contacts()

In [5]:
ts.st_init_v()

ts.sx_search_v()
ts.st_montag_B()

# Питания
ts.sx_search_sh_circle()
ts.st_init_shina()
ts.st_montag_shina()

IndexError: single positional indexer is out-of-bounds

In [6]:
i = 13
df1 = ts.text_tools_rcn.df_match('^[1-3]-\d{2,3}')
ref1 = df1.iloc[i]['ref']
t_loc = df1.iloc[i]['text']
dt_name = ts.text_tools_rcn.find_text_ref(ref1, 8, 8, 0, 5, lst_exclude=ts.ls_contact)

In [4]:
df1.iloc[i]

text           1-57
x       1473.699057
y         95.793257
ref      1073815566
Name: 622, dtype: object

In [7]:
t_name = dt_name['text']
dc_name = ts.text_tools_rcn.find_cirle_ref(dt_name['ref'], 20, 20, 8, 9, radius_gt=3)
if dc_name is not None:
    dt_contact = ts.text_tools_rcn.find_text(dc_name['x'], dc_name['y'], 10, 10, 10, 10,
                                               lst_filter=ts.ls_contact)
else:
    dt_contact = ts.text_tools_rcn.find_text_ref(dt_name['ref'], 20, 20, 0, 15,
                                                   lst_filter=ts.ls_contact)

In [23]:
df_contact = dt_contact['df']

In [24]:
ds = ts._direction_contact(df_contact)

In [25]:
group = t_loc + ';' + ':'.join(df_contact['text'].values.tolist())

In [27]:
j = 0
ls = []
t_contact = df_contact.iloc[j]['text']
t_ref = df_contact.iloc[j]['ref']
t_x = df_contact.iloc[j]['x']
t_y = df_contact.iloc[j]['y']
st, p, r = ts.get_loc(t_loc)
ls.append([t_loc, st, p, r, t_name, t_contact, t_ref, t_x, t_y, group, ds[t_contact][0],
           ds[t_contact][1]])

In [ ]:
ref1 = df1.iloc[i]['ref']
            t_loc = df1.iloc[i]['text']
            dt_name = self.text_tools_rcn.find_text_ref(ref1, 8, 8, 0, 5, lst_exclude=self.ls_contact)
            if dt_name is not None:
                t_name = dt_name['text']
                dc_name = self.text_tools_rcn.find_cirle_ref(dt_name['ref'], 20, 20, 8, 9, radius_gt=3)
                if dc_name is not None:
                    dt_contact = self.text_tools_rcn.find_text(dc_name['x'], dc_name['y'], 8, 8, 8, 8,
                                                               lst_filter=self.ls_contact)
                else:
                    dt_contact = self.text_tools_rcn.find_text_ref(dt_name['ref'], 10, 10, 0, 15,
                                                                   lst_filter=self.ls_contact)

                if dt_contact is not None:
                    df9 = dt_contact
                    df_contact = dt_contact['df']
                    if df_contact['text'].tolist() == ['1', '2', '3', '4']:
                        df_contact = df_contact[df_contact['text'].isin(['2', '4'])]
                    ds = self._direction_contact(df_contact)
                    group = t_loc + ';' + ':'.join(df_contact['text'].values.tolist())

                    for j in range(len(df_contact)):
                        t_contact = df_contact.iloc[j]['text']
                        t_ref = df_contact.iloc[j]['ref']
                        t_x = df_contact.iloc[j]['x']
                        t_y = df_contact.iloc[j]['y']
                        st, p, r = self.get_loc(t_loc)
                        ls.append([t_loc, st, p, r, t_name, t_contact, t_ref, t_x, t_y, group, ds[t_contact][0],
                                   ds[t_contact][1]])

In [67]:
ls_st_name = ['1', '2', '3']
ls_st_sh_name = ['М', 'П', 'МСХ', 'СХМ']

In [ ]:
def filter_df(df, col1_val1, col2_val2=None, col3_val3=None):
    col1, val1 = col1_val1
    df1 = df[df[col1] == val1]
    if col2_val2 is not None:
        col2, val2 = col2_val2
        df1 = df1[df1[col2] == val2]

    if col3_val3 is not None:
        col3, val3 = col3_val3
        df1 = df1[df1[col3] == val3]
    return df1

In [95]:
df5 = pd.DataFrame(columns=['st', 'sh', 'loc', 'contact', 'polka'])

In [97]:
df6 = filter_df(df5, ('st', '1'), ('sh', 'М'), ('polka', '1'))
if len(df6) == 0:
    df5 = df5.append({'st': '1', 'sh': 'М', 'loc': '1-1', 'contact': '1', 'polka': '1'}, ignore_index=True)
else:

In [100]:
df5 = df5.append({'st': '1', 'sh': 'М', 'loc': '1-1', 'contact': '2', 'polka': '1'}, ignore_index=True)

In [101]:
df6 = filter_df(df5, ('st', '1'), ('sh', 'М'), ('polka', '1'))
df6

,st,sh,loc,contact,polka
0,1,М,1-1,1,1
1,1,М,1-1,2,1


In [103]:
cont = int(df6.tail(1)['contact'].values[0]) + 1

3

In [110]:
df = pd.DataFrame(columns=['st', 'sh', 'loc', 'contact', 'polka'])

def st_get_contact_sh(st,  polka, sh, loc):
    global df
    df1 = filter_df(df, ('st', st), ('sh', sh), ('polka', polka))
    if len(df1) == 0:
        df = df.append({'st': st, 'sh': sh, 'loc': loc, 'contact': '1', 'polka': polka}, ignore_index=True)
        return '1'
    else:
        cont = str(int(df1.tail(1)['contact'].values[0]) + 1)
        df = df.append({'st': st, 'sh': sh, 'loc': loc, 'contact': cont, 'polka': polka}, ignore_index=True)
        return cont
for st_name in ls_st_name:
    for sh_name in ls_st_sh_name:
        # df1 = ts.df_sx_contacts[(ts.df_sx_contacts['st'] == st_name) & (ts.df_sx_contacts['pit'] == sh_name)]
        df1 = filter_df(ts.df_sx_contacts, ('st', st_name), ('pit', sh_name))
        if len(df1) > 0:
            df1 = df1.sort_values(by=['p', 'r', 'contact'])
            for i in range(len(df1)):
                loc = df1['loc'].values[i]
                contact = df1['contact'].values[i]
                polka = str(ts.get_nearest_sh(loc, sh_name))
                cont_sh = st_get_contact_sh(st_name,  polka, sh_name, loc)

                p = df1['p'].values[i]
                r = df1['r'].values[i]
                if p == polka:
                    loc_to = f'{polka}Ш-{cont_sh}'
                    loc_to_sh = f'{r}-{contact}'
                else:
                    loc_to = f'Ш-{cont_sh}'
                    loc_to_sh = f'{p}{r}-{contact}'

                ts.add_st_contact(loc, contact, [loc_to, sh_name])


                ts.add_st_sh(st_name, f'{polka}Ш-{sh_name}', cont_sh, loc_to_sh)



In [91]:
filter_df(ts.df_sx_contacts, ('st', '1'), ('pit', 'М'))

,loc,st,p,r,name,contact,ref,x,y,group,direction,side,b1,b2,l1,l2,pit
2,1-13,1,1,3,Зв9,41,1073774584,1049.418926,126.295838,1-13;41:42,horizontal,left,None,None,None,None,М
5,1-13,1,1,3,Зв9,4,1073774728,1276.972272,-76.954656,1-13;1:4,horizontal,right,None,None,None,None,М
8,1-27,1,2,7,ЧПИ13,41,1073775018,1675.420362,88.295921,1-27;41:42,horizontal,left,None,None,None,None,М
17,1-27,1,2,7,ЧПИ13,4,1073775262,1634.973193,-31.954275,1-27;1:4,horizontal,right,None,None,None,None,М
19,1-26,1,2,6,ДСН13,4,1073775187,1634.973193,-47.954536,1-26;1:4,horizontal,right,None,None,None,None,М
25,1-28,1,2,8,НПИ13,4,1073775291,1634.973193,-15.954358,1-28;1:4,horizontal,right,None,None,None,None,М
32,1-28,1,2,8,НПИ13,41,1073775484,1676.420497,128.296056,1-28;41:42,horizontal,left,None,None,None,None,М
39,1-12,1,1,2,Зв5,4,1073776084,3166.972682,-76.954656,1-12;1:4,horizontal,right,None,None,None,None,М
42,1-17,1,1,7,Зв6,41,1073776348,3359.419746,126.295838,1-17;41:42,horizontal,left,None,None,None,None,М
45,1-17,1,1,7,Зв6,4,1073776474,3586.973092,-76.954656,1-17;1:4,horizontal,right,None,None,None,None,М


In [76]:
df1 = ts.df_sx_contacts[(ts.df_sx_contacts['st'] == '1') &(ts.df_sx_contacts['pit'] == sh_name)]

In [78]:
df2 = ts.df_st_sh_p_ls[(ts.df_st_sh_p_ls['st'] == '1') &(ts.df_st_sh_p_ls['pit'] == sh_name)]
df2

,st,pit,p
0,1,М,[3]


In [79]:
ts.get_val_df_first(df2, 'p')

['3']

In [81]:
loc = ts.df_sx_contacts.iloc[0]['loc']
loc

'1-13'

In [82]:
pit = ts.df_sx_contacts.iloc[0]['pit']
pit

'П'

In [85]:
ts.get_nearest_sh(loc, pit)

4

In [88]:
df2 = df1.sort_values(by=['p', 'r', 'contact'])

In [89]:
x1, x2 = (1,2)

In [131]:

df_st_b = init_st_b()

In [132]:
df_contacts = search_contacts(text_tools_rcn)
df_contacts[['b1', 'b2', 'l1', 'l2', 'pit']] = None
df_contacts

,loc,st,p,r,name,contact,ref,x,y,group,direction,side,b1,b2,l1,l2,pit
0,1-13,1,1,3,Зв9,21,1073761402,1048.348584,142.296106,1-13;21:22,horizontal,left,None,None,None,None,None
1,1-13,1,1,3,Зв9,22,1073761405,1059.439607,142.296106,1-13;21:22,horizontal,right,None,None,None,None,None
2,1-13,1,1,3,Зв9,41,1073761400,1049.418926,126.295838,1-13;41:42,horizontal,left,None,None,None,None,None
3,1-13,1,1,3,Зв9,42,1073761401,1060.510302,126.295838,1-13;41:42,horizontal,right,None,None,None,None,None
4,1-13,1,1,3,Зв9,1,1073761542,1266.872191,-76.822717,1-13;1:4,horizontal,left,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1-21,1,2,1,Зв10,22,1073766462,4209.440188,142.295573,1-21;21:22,horizontal,right,None,None,None,None,None
92,1-21,1,2,1,Зв10,41,1073766457,4199.419506,126.295306,1-21;41:42,horizontal,left,None,None,None,None,None
93,1-21,1,2,1,Зв10,42,1073766458,4210.510883,126.295306,1-21;41:42,horizontal,right,None,None,None,None,None
94,1-21,1,2,1,Зв10,1,1073766468,4416.872252,-76.822795,1-21;1:4,horizontal,left,None,None,None,None,None


In [133]:
df_B = text_tools_rcn.df_match('^[1-3]-В')
df_B = df_B.apply(rep_xy_on_circle, axis=1)
df_101 = text_tools_rcn.df_match('^10[1-5]-')
df_101 = df_101.apply(rep_xy_on_circle, axis=1)
df_C = text_tools_rcn.df_match('^C[1-8]:')
df_C = df_C.apply(rep_xy_on_circle, axis=1)

# отсортировать по x возрастанию потом y по убыванию
df_B = df_B.sort_values(by=['x', 'y'], key=lambda x: x.map(dict(zip(x, range(len(x))))))

df_shina = None

In [134]:

def f1():
    for i in range(len(df_B)):
        res = df_B.iloc[i]
        df1, df_name1 = find_link_by_b(res['x'], res['y'], 10, 0, 5, 5, side='left')
        df2, df_name2 = find_link_by_b(res['x'], res['y'], 0, 0, 5, 5, side='right')
        loc = None
        contact = None
        df_name_st = None
        df_name_no_st = None
        if df_name1 == 'df_contacts':
            loc = df1.iloc[0]['loc']
            df_cont = df1
            contact = df1.iloc[0]['contact']
            b1_dev = df1.iloc[0]['b1']
            b2_dev = df1.iloc[0]['b2']
            df_name_st = df_name1
        if df_name1 == 'df_101' or df_name1 == 'df_C':
            df_no_cont = df1
            b_st = df1.iloc[0]['text']
            df_name_no_st = df_name1

        if df_name2 == 'df_contacts':
            loc = df2.iloc[0]['loc']
            df_cont = df2
            contact = df2.iloc[0]['contact']
            b1_dev = df2.iloc[0]['b1']
            b2_dev = df2.iloc[0]['b2']
            df_name_st = df_name2
        if df_name2 == 'df_101' or df_name2 == 'df_C':
            df_no_cont = df2
            b_st = df2.iloc[0]['text']
            df_name_no_st = df_name2

        if loc is not None and df_name_no_st is not None:
            st, p, r = get_loc(loc)
            ls = ds_st_b[df_name_no_st]
            df_t = df_st_b[(df_st_b['st']==st) & (df_st_b['b'].isin(ls)) & (df_st_b['l1']=='')]
            df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
            df_bb = df_t.head(1)
            b, n = df_bb['b'].values[0],  df_bb['n'].values[0]    #'B110', 1
            s_b = f'{st}-{b}-{n}'

            text_tools_rcn.replace(res['ref'], s_b)
            add_st_contact(loc=loc, contact=contact, loc_to=[f'{b}-{n}'])
            add_st_v(st, b, str(n), [f'{p}{r}-{contact}', b_st])

            df_st_b.loc[df_bb.index, 'l1'] =  loc
            df_st_b.loc[df_bb.index, 'l2'] =  b_st

            if df_cont.iloc[0]['b1'] is None:
                df_contacts.loc[df_cont.index, 'b1'] =  f'{b}-{n}'
            else:
                df_contacts.loc[df_cont.index, 'b2'] =  f'{b}-{n}'

        if loc is None and df_name_no_st is not None:
            st = res['text'][0]
            ls = ds_st_b[df_name_no_st]
            df_t = df_st_b[(df_st_b['st']==st) & (df_st_b['b'].isin(ls)) & (df_st_b['l1']=='')]
            df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
            df_bb = df_t.head(1)
            b, n = df_bb['b'].values[0],  df_bb['n'].values[0]    #'B110', 1
            s_b = f'{st}-{b}-{n}'

            text_tools_rcn.replace(res['ref'], s_b)
            # add_st_contact(loc=loc, contact=contact, loc_to=[f'{b}-{n}'])
            add_st_v(st, b, str(n), ['', b_st])

            df_st_b.loc[df_bb.index, 'l1'] =  'резерв'
            df_st_b.loc[df_bb.index, 'l2'] =  b_st


In [135]:
f1()

In [34]:
text_tools = text_tools_st[f'c{2}']

In [28]:
df_sh = text_tools_rcn.df_match(r'[1-3]-Ш')
df_sh

,text,x,y,ref
12,1-Ш,1017.483783,150.418925,1073761392
16,1-Ш,1017.483938,134.418657,1073761396
167,1-Ш,1312.743418,-68.581717,1073761547
732,1-Ш,1651.744050,-7.581766,1073762114
916,1-Ш,1644.485496,136.418512,1073762300
922,1-Ш,1630.896617,151.418661,1073762306
925,1-Ш,1644.485284,152.418795,1073762309
5090,1-Ш,2487.483783,150.418925,1073766474
5092,1-Ш,2487.483938,134.418657,1073766476
5094,1-Ш,2782.743418,-68.581717,1073766478


In [60]:
ref1 = df_sh.iloc[0]['ref']

In [72]:
df_sh = text_tools_rcn.df_match(r'[1-3]-Ш')

In [ ]:
ds = text_tools_rcn.find_text_ref(ref1, 6, 6, 0, 5)

In [136]:
def search_sh(df_shina=None):
    df_sh = text_tools_rcn.df_match(r'[1-3]-Ш')

    for i in range(len(df_sh)):
        ref1 = df_sh.iloc[i]['ref']
        ds = text_tools_rcn.find_text_ref(ref1, 6, 6, 0, 5)
        if ds is not None:
            name_sh = ds['text'] # 'П'
            ds1 = text_tools_rcn.find_cirle_ref(ds['ref'], 5, 5, 0, 10, radius_lt=2)
            if ds1 is not None:
                df2 = ds1['df']
                df2['pit'] = name_sh

            ds_lines = text_tools_rcn.find_line(ds['x'], ds['y'], 5, 5, 0, 10)

            if ds_lines is not None:
                df_lines = ds_lines['df']

                is_line = False
                for i in range(len(df_lines)):
                    res = df_lines.iloc[i]
                    if res['x1'] == res['x2']:
                        is_line = True
                        break
                if is_line:
                    df2 = text_tools_rcn.find_cirle_ref(ds['ref'], 5, 5, 0, 180, radius_lt=2)['df']
                    df2['pit'] = name_sh

            if df_shina is None:
                df_shina = df2
            else:
                df_shina = df_shina.append(df2)

    return df_shina

In [137]:
df_shina_cirle = search_sh()
df_shina_cirle

,x,y,radius,ref,pit
0,1021.054485,142.729975,0.749852,1073766619,П
1,1021.054485,126.729835,0.750078,1073766620,М
26,1314.055125,-76.270540,0.750078,1073766645,М
115,1653.055757,-15.270591,0.750078,1073766734,М
116,1653.055757,-31.270506,0.750078,1073766735,М
117,1653.055757,-47.270421,0.750078,1073766736,М
121,1648.056053,128.729700,0.750078,1073766740,М
103,1633.055913,143.730067,0.749852,1073766722,П
104,1633.055913,127.730152,0.749852,1073766723,П
105,1633.055913,111.729885,0.749852,1073766724,П


In [144]:
def create_shina_stativ():
    df5 = text_tools_st['c1'].df_match(r'.Ш-[А-Я]')
    df5 = df5.reset_index(drop=True)
    df5['st'] = '1'
    df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
    df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
    df_shina_stativ = df5.copy()

    df5 = text_tools_st['c2'].df_match(r'.Ш-[А-Я]')
    df5 = df5.reset_index(drop=True)
    df5['st'] = '2'
    df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
    df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
    df_shina_stativ = df_shina_stativ.append(df5)

    df5 = text_tools_st['c3'].df_match(r'.Ш-[А-Я]')
    df5 = df5.reset_index(drop=True)
    df5['st'] = '3'
    df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
    df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
    df_shina_stativ = df_shina_stativ.append(df5)

    df_shina_stativ = df_shina_stativ.reset_index(drop=True)

    return df_shina_stativ


def set_value_df(df, ref, col, value):
    df.loc[df['ref'] == ref, col] = value
    return df


def set_pit_contact(df_contacts, df_shina):
    df1 = df_contacts[df_contacts['b1'].isnull()]
    for i in range(len(df1)):
        res = df1.iloc[i]
        ref1 = res['ref']
        df2 = find_link_by_sh(df_shina, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])
        if df2 is not None:
            df3 = df2['df']
            pit = df3['pit'].values[0]

            df_contacts = set_value_df(df_contacts, ref1, 'pit', pit)


In [ ]:
df_shina_stativ = create_shina_stativ()
set_pit_contact(df_contacts, df_shina_cirle)
df_contacts

In [146]:
df_shina_stativ

,text,x,y,ref,st,p,pit
0,3Ш-М,4188.824734,45.834429,1073742127,1,3,М
1,4Ш-П,4608.824734,45.834429,1073742162,1,4,П
2,5Ш-СХМ,5032.793484,45.834429,1073742197,1,5,СХМ
3,6Ш-МСХ,5452.793484,45.834429,1073742232,1,6,МСХ
4,2Ш-П,3763.963016,43.189257,1073754204,2,2,П
5,4Ш-П,4603.963016,43.189257,1073754209,2,4,П
6,6Ш-П,5443.963016,43.189257,1073754210,2,6,П
7,8Ш-П,6283.963016,43.189257,1073754211,2,8,П
8,2Ш-П,3763.963016,43.189257,1073746376,3,2,П
9,4Ш-П,4603.963016,43.189257,1073746381,3,4,П


In [147]:
df_contacts.head(3)

,loc,st,p,r,name,contact,ref,x,y,group,direction,side,b1,b2,l1,l2,pit
0,1-13,1,1,3,Зв9,21,1073761402,1048.348584,142.296106,1-13;21:22,horizontal,left,None,None,None,None,П
1,1-13,1,1,3,Зв9,22,1073761405,1059.439607,142.296106,1-13;21:22,horizontal,right,В19-1,None,None,None,None
2,1-13,1,1,3,Зв9,41,1073761400,1049.418926,126.295838,1-13;41:42,horizontal,left,None,None,None,None,М


In [150]:
df_contacts[(df_contacts['st'] == '1') & (df_contacts['pit'] == 'М')]

,loc,st,p,r,name,contact,ref,x,y,group,direction,side,b1,b2,l1,l2,pit
2,1-13,1,1,3,Зв9,41,1073761400,1049.418926,126.295838,1-13;41:42,horizontal,left,None,None,None,None,М
5,1-13,1,1,3,Зв9,4,1073761544,1276.972272,-76.954656,1-13;1:4,horizontal,right,None,None,None,None,М
8,1-27,1,2,7,ЧПИ13,41,1073761834,1675.420362,88.295921,1-27;41:42,horizontal,left,None,None,None,None,М
17,1-27,1,2,7,ЧПИ13,4,1073762078,1634.973193,-31.954275,1-27;1:4,horizontal,right,None,None,None,None,М
19,1-26,1,2,6,ДСН13,4,1073762003,1634.973193,-47.954536,1-26;1:4,horizontal,right,None,None,None,None,М
25,1-28,1,2,8,НПИ13,4,1073762109,1634.973193,-15.954358,1-28;1:4,horizontal,right,None,None,None,None,М
32,1-28,1,2,8,НПИ13,41,1073762304,1676.420497,128.296056,1-28;41:42,horizontal,left,None,None,None,None,М
39,1-12,1,1,2,Зв5,4,1073762904,3166.972682,-76.954656,1-12;1:4,horizontal,right,None,None,None,None,М
42,1-17,1,1,7,Зв6,41,1073763168,3359.419746,126.295838,1-17;41:42,horizontal,left,None,None,None,None,М
45,1-17,1,1,7,Зв6,4,1073763294,3586.973092,-76.954656,1-17;1:4,horizontal,right,None,None,None,None,М


In [152]:
df_shina_stativ[(df_shina_stativ['st'] == '2') & (df_shina_stativ['pit'] == 'П')]

,text,x,y,ref,st,p,pit
4,2Ш-П,3763.963016,43.189257,1073754204,2,2,П
5,4Ш-П,4603.963016,43.189257,1073754209,2,4,П
6,6Ш-П,5443.963016,43.189257,1073754210,2,6,П
7,8Ш-П,6283.963016,43.189257,1073754211,2,8,П


In [160]:
df_ls_sh_p =  df_shina_stativ.groupby(['st', 'pit'])['p'].agg(lambda x: list(x)).reset_index()
df_ls_sh_p

,st,pit,p
0,1,М,[3]
1,1,МСХ,[6]
2,1,П,[4]
3,1,СХМ,[5]
4,2,П,"[2, 4, 6, 8]"
5,3,П,"[2, 4, 6, 8]"


In [193]:
df_shina_stativ

,text,x,y,ref,st,p,pit
0,3Ш-М,4188.824734,45.834429,1073742127,1,3,М
1,4Ш-П,4608.824734,45.834429,1073742162,1,4,П
2,5Ш-СХМ,5032.793484,45.834429,1073742197,1,5,СХМ
3,6Ш-МСХ,5452.793484,45.834429,1073742232,1,6,МСХ
4,2Ш-П,3763.963016,43.189257,1073754204,2,2,П
5,4Ш-П,4603.963016,43.189257,1073754209,2,4,П
6,6Ш-П,5443.963016,43.189257,1073754210,2,6,П
7,8Ш-П,6283.963016,43.189257,1073754211,2,8,П
8,2Ш-П,3763.963016,43.189257,1073746376,3,2,П
9,4Ш-П,4603.963016,43.189257,1073746381,3,4,П


In [164]:
ls1 = df_ls_sh_p[(df_ls_sh_p['st'] == '2') & (df_ls_sh_p['pit'] == 'П')]['p'].values[0]
ls1 = [int(x) for x in ls1]
ls1

['2', '4', '6', '8']

In [ ]:
df_shina_cirle = search_sh()

In [191]:
class Tools:
    def __init__(self, text_tools_st, text_tools_rcn):
        self.df = None
        self.text_tools_st = text_tools_st
        self.text_tools_rcn = text_tools_rcn

        self.ls_contact = ['1', '2', '3', '4', '11', '12', '13', '21', '22', '23', '31', '32', '33', '41', '42', '43', '51', '52', '53', '61', '62', '63', '71', '72', '73', '81', '82', '83', '111', '112', '113', '121', '122', '123']

        self.ds_st_b = {
            'df_101': [  'В19', 'В110', 'В111', 'В112',  'В29', 'В210', 'В211', 'В212', 'В18','В28',],
            'df_C': ['В16', 'В17',  'В26', 'В27', 'В25'],
            'df_contacts': ['В18','В28'],
        }
        self.df_st_b = None

        self.df_st_shina = None
        self.df_st_sh_p_ls = None
        self.df_B = None
        self.df_101 = None
        self.df_C = None

        self.df_sx_sh_circle = None
        self.df_sx_contacts = None


    def sx_search_contacts(self):
        ls = []
        df1 = self.text_tools_rcn.df_match('^[1-3]-\d{2,3}')

        for i in range(len(df1)):
            ref1 = df1.iloc[i]['ref']
            t_loc = df1.iloc[i]['text']
            dt_name = self.text_tools_rcn.find_text_ref(ref1, 8, 8, 0, 5, lst_exclude=self.ls_contact)
            if dt_name is not None:
                t_name = dt_name['text']
                dc_name = self.text_tools_rcn.find_cirle_ref(dt_name['ref'], 20, 20, 8, 9, radius_gt=3)
                if dc_name is not None:
                    dt_contact = self.text_tools_rcn.find_text(dc_name['x'], dc_name['y'], 8, 8, 8, 8, lst_filter=self.ls_contact)
                else:
                    dt_contact = self.text_tools_rcn.find_text_ref(dt_name['ref'], 10, 10, 0, 15, lst_filter=self.ls_contact)

                if dt_contact is not None:
                    df_contact = dt_contact['df']
                    if df_contact['text'].tolist() == ['1', '2', '3', '4']:
                        df_contact = df_contact[df_contact['text'].isin(['2', '4'])]
                    ds = self._direction_contact(df_contact)
                    group = t_loc + ';' + ':'.join(df_contact['text'].values.tolist())

                    for j in range(len(df_contact)):
                        t_contact = df_contact.iloc[j]['text']
                        t_ref = df_contact.iloc[j]['ref']
                        t_x = df_contact.iloc[j]['x']
                        t_y = df_contact.iloc[j]['y']
                        st, p, r = self.get_loc(t_loc)
                        ls. append([t_loc, st, p, r, t_name, t_contact, t_ref, t_x, t_y, group, ds[t_contact][0], ds[t_contact][1]])


        df2 = pd.DataFrame(ls, columns=['loc', 'st', 'p', 'r', 'name', 'contact', 'ref', 'x', 'y', 'group', 'direction', 'side'])
        return df2

    def sx_search_v(self):
        self.df_B = self.text_tools_rcn.df_match('^[1-3]-В')
        self.df_B = self.df_B.apply( self._rep_xy_on_circle, axis=1)
        self.df_101 = self.text_tools_rcn.df_match('^10[1-5]-')
        self.df_101 = self.df_101.apply( self._rep_xy_on_circle, axis=1)
        self.df_C = self.text_tools_rcn.df_match('^C[1-8]:')
        self.df_C = self.df_C.apply( self._rep_xy_on_circle, axis=1)

        # отсортировать по x возрастанию потом y по убыванию
        self.df_B = self.df_B.sort_values(by=['x', 'y'], key=lambda x: x.map(dict(zip(x, range(len(x))))))

    def sx_search_sh_circle(self):
        df_sh = self.text_tools_rcn.df_match(r'[1-3]-Ш')

        for i in range(len(df_sh)):
            ref1 = df_sh.iloc[i]['ref']
            ds = self.text_tools_rcn.find_text_ref(ref1, 6, 6, 0, 5)
            if ds is not None:
                df2 = None
                name_sh = ds['text'] # 'П'
                ds1 = self.text_tools_rcn.find_cirle_ref(ds['ref'], 5, 5, 0, 10, radius_lt=2)
                if ds1 is not None:
                    df2 = ds1['df']
                    df2['pit'] = name_sh

                ds_lines = self.text_tools_rcn.find_line(ds['x'], ds['y'], 5, 5, 0, 10)

                if ds_lines is not None:
                    df_lines = ds_lines['df']

                    is_line = False
                    for i in range(len(df_lines)):
                        res = df_lines.iloc[i]
                        if res['x1'] == res['x2']:
                            is_line = True
                            break
                    if is_line:
                        df2 = self.text_tools_rcn.find_cirle_ref(ds['ref'], 5, 5, 0, 180, radius_lt=2)['df']
                        df2['pit'] = name_sh

                if df2 is not None:
                    if self.df_sx_sh_circle is None:
                        self.df_sx_sh_circle = df2.copy()
                    else:
                        self.df_sx_sh_circle = self.df_sx_sh_circle.append(df2)

        return df_shina

    def sx_set_pit_contact(self):
        df1 = self.df_sx_contacts[self.df_sx_contacts['b1'].isnull()]
        for i in range(len(df1)):
            res = df1.iloc[i]
            ref1 = res['ref']
            df2 = self._find_link_by_sh(self.df_sx_sh_circle, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])
            if df2 is not None:
                df3 = df2['df']
                pit = df3['pit'].values[0]

                self.df_sx_contacts = set_value_df(self.df_sx_contacts, ref1, 'pit', pit)


    def st_init_v(self):
        ls_B = ['В15', 'В25', 'В16', 'В26', 'В17', 'В27', 'В18', 'В28', 'В19', 'В29', 'В110', 'В111', 'В112', 'В210', 'В211', 'В212']
        ls_st = ['1', '2', '3']
        ls = []
        for st in ls_st:
            for b in ls_B:
                for i in range(1, 21):
                    ls.append([st, b, i])
        self.df_st_b = pd.DataFrame(ls, columns=['st', 'b', 'n'])
        self.df_st_b[['l1', 'l2', 'p']] = ''

    def st_init_shina(self):
        df5 = self.text_tools_st['c1'].df_match(r'.Ш-[А-Я]')
        df5 = df5.reset_index(drop=True)
        df5['st'] = '1'
        df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
        df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
        self.df_st_shina = df5.copy()

        df5 = self.text_tools_st['c2'].df_match(r'.Ш-[А-Я]')
        df5 = df5.reset_index(drop=True)
        df5['st'] = '2'
        df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
        df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
        self.df_st_shina = self.df_st_shina.append(df5)

        df5 = self.text_tools_st['c3'].df_match(r'.Ш-[А-Я]')
        df5 = df5.reset_index(drop=True)
        df5['st'] = '3'
        df5['p'] = df5['text'].apply(lambda x: x.split('Ш-')[0])
        df5['pit'] = df5['text'].apply(lambda x: x.split('Ш-')[1])
        self.df_st_shina = self.df_st_shina.append(df5)

        self.df_st_shina = self.df_st_shina.reset_index(drop=True)

        self.df_st_sh_p_ls =  self.df_st_shina.groupby(['st', 'pit'])['p'].agg(lambda x: list(x)).reset_index()

    def st_montag_B(self):
        for i in range(len(self.df_B)):
            res = self.df_B.iloc[i]
            df1, df_name1 = self._find_link_by_b(res['x'], res['y'], 10, 0, 5, 5, side='left')
            df2, df_name2 = self._find_link_by_b(res['x'], res['y'], 0, 0, 5, 5, side='right')
            loc = None
            contact = None
            df_name_st = None
            df_name_no_st = None
            if df_name1 == 'df_contacts':
                loc = df1.iloc[0]['loc']
                df_cont = df1
                contact = df1.iloc[0]['contact']
                b1_dev = df1.iloc[0]['b1']
                b2_dev = df1.iloc[0]['b2']
                df_name_st = df_name1
            if df_name1 == 'df_101' or df_name1 == 'df_C':
                df_no_cont = df1
                b_st = df1.iloc[0]['text']
                df_name_no_st = df_name1

            if df_name2 == 'df_contacts':
                loc = df2.iloc[0]['loc']
                df_cont = df2
                contact = df2.iloc[0]['contact']
                b1_dev = df2.iloc[0]['b1']
                b2_dev = df2.iloc[0]['b2']
                df_name_st = df_name2
            if df_name2 == 'df_101' or df_name2 == 'df_C':
                df_no_cont = df2
                b_st = df2.iloc[0]['text']
                df_name_no_st = df_name2

            if loc is not None and df_name_no_st is not None:
                st, p, r = self.get_loc(loc)
                ls = self.ds_st_b[df_name_no_st]
                df_t = self.df_st_b[(self.df_st_b['st']==st) & (self.df_st_b['b'].isin(ls)) & (self.df_st_b['l1']=='')]
                df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
                df_bb = df_t.head(1)
                b, n = df_bb['b'].values[0],  df_bb['n'].values[0]    #'B110', 1
                s_b = f'{st}-{b}-{n}'

                self.text_tools_rcn.replace(res['ref'], s_b)
                self.add_st_contact(loc=loc, contact=contact, loc_to=[f'{b}-{n}'])
                self.add_st_v(st, b, str(n), [f'{p}{r}-{contact}', b_st])

                self.set_val_df(df_name_st, df_bb['ref'], 'l1', loc)
                self.set_val_df(df_name_st, df_bb['ref'], 'l2', b_st)

                if self.get_val_df_first(df_cont, 'b1') is None:
                    self.set_val_df(self.df_sx_contacts, df_cont['ref'], 'b1', f'{b}-{n}')
                    # self.df_sx_contacts.loc[df_cont.index, 'b1'] =  f'{b}-{n}'
                else:
                    self.set_val_df(self.df_sx_contacts, df_cont['ref'], 'b2', f'{b}-{n}')
                    # self.df_sx_contacts.loc[df_cont.index, 'b2'] =  f'{b}-{n}'

            if loc is None and df_name_no_st is not None:
                st = res['text'][0]
                ls = self.ds_st_b[df_name_no_st]
                df_t = self.df_st_b[(self.df_st_b['st']==st) & (self.df_st_b['b'].isin(ls)) & (self.df_st_b['l1']=='')]
                df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
                df_bb = df_t.head(1)
                b, n = df_bb['b'].values[0],  df_bb['n'].values[0]    #'B110', 1
                s_b = f'{st}-{b}-{n}'

                self.text_tools_rcn.replace(res['ref'], s_b)
                self.add_st_v(st, b, str(n), ['', b_st])

                self.set_val_df(df_name_st, df_bb['ref'], 'l1', 'резерв')
                self.set_val_df(df_name_st, df_bb['ref'], 'l2', b_st)





    def get_nearest_sh(self, loc, pit):
        st, p, r = self.get_loc(loc)
        ls1 = self.df_st_sh_p_ls[(self.df_st_sh_p_ls['st'] == st) & (self.df_st_sh_p_ls['pit'] == pit)]['p']
        if len(ls1) > 0:
            ls1 = ls1.values[0]
            ls1 = [int(x) for x in ls1]
            return self._find_nearest(ls1, int(p))

    def add_st_contact(self, loc, contact, loc_to):

        st, p, r = self.get_loc(loc)

        text_tools = self.text_tools_st[st]

        df = text_tools.doc.df_texts

        df1 = df[df['text'] == f'Полка {p}']
        ref1 = df1.iloc[0]['ref']
        d1 = text_tools.find_text_ref(ref1, 360, 30, 270, -250)
        df1 = d1['df']

        df1 = df1[df1['text'] == r]
        ref1 = df1.iloc[0]['ref']
        d1 = text_tools.find_text_ref(ref1, 25, -15, 0, 240)
        df1 = d1['df']

        df1 = df1[df1['text'] == contact]
        ds = df1.iloc[0].to_dict()

        if len(loc_to) == 1:
            text_tools.add_text(loc_to[0], ds['x'] + 15, ds['y'], ref_text_font=ds['ref'])
        elif len(loc_to) == 2:
            text_tools.add_text(loc_to[0], ds['x'] + 15, ds['y'], ref_text_font=ds['ref'])
            text_tools.add_text(loc_to[1], ds['x'] + 35, ds['y'], ref_text_font=ds['ref'])
        elif len(loc_to) == 3:
            text_tools.add_text(loc_to[0], ds['x'] + 15, ds['y'] + 2, ref_text_font=ds['ref'])
            text_tools.add_text(loc_to[1], ds['x'] + 15, ds['y'] - 2, ref_text_font=ds['ref'])
            text_tools.add_text(loc_to[2], ds['x'] + 35, ds['y'], ref_text_font=ds['ref'])

        return True

    def add_st_v(self, st, v, contact, loc_to):

        text_tools = self.text_tools_st[st]

        df_v = text_tools.df_match(v)
        if len(df_v) == 0:
            return False

        ref_v = df_v.iloc[0]['ref']

        d1 = text_tools.find_text_ref(ref_v, 7, 7, 0, 85)
        df1 = d1['df']

        df1 = df1[df1['text'] == contact]
        ds = df1.iloc[0].to_dict()

        if int(contact) % 2 == 1:
            if len(loc_to) == 1:
                text_tools.add_text(loc_to[0], ds['x'] - 12, ds['y'], ref_text_font=ds['ref'])
            elif len(loc_to) == 2:
                text_tools.add_text(loc_to[0], ds['x'] - 12, ds['y'], ref_text_font=ds['ref'])
                text_tools.add_text(loc_to[1], ds['x'] - 12, ds['y'] - 4, ref_text_font=ds['ref'])
        else:
            if len(loc_to) == 1:
                text_tools.add_text(loc_to[0], ds['x'] + 12, ds['y'], ref_text_font=ds['ref'])
            elif len(loc_to) == 2:
                text_tools.add_text(loc_to[0], ds['x'] + 12, ds['y'], ref_text_font=ds['ref'])
                text_tools.add_text(loc_to[1], ds['x'] + 12, ds['y'] - 4, ref_text_font=ds['ref'])

    def add_st_sh(self, st, sh_st, contact, loc_to):

        text_tools = self.text_tools_st[st]

        df1 = text_tools.df_match(sh_st)
        ref1 = df1.iloc[0]['ref']
        d1 = text_tools.find_text_ref(ref1, 90, 110, -25, 40)
        df1 = d1['df']
        df1 = df1[df1['text'] == str(contact)]
        ds = df1.iloc[0].to_dict()

        text_tools.add_text(loc_to, ds['x'] + 1, ds['y'] + 8, ref_text_font=ds['ref'], rotation=90)

        return True



    # выбрать из списка ls1 число ближйшее к заданному
    def _find_nearest(self, ls, value):
        ls = np.array(ls)
        idx = (np.abs(ls - value)).argmin()
        return ls[idx]

    def _rep_xy_on_circle(self, row):
        res =  self.text_tools_rcn.find_cirle(row['x'], row['y'], 10, 10, 0, 8, radius_lt=2)
        if res is not None:
            row['x'] = res['x']
            row['y'] = res['y']
        return row

    def _find_link_by_b(self, x, y, dx_left=1, dx_right=1, dy_top=1, dy_bottom=1, side='left'):
        df_name = 'df_contacts'
        res1 = None
        for i in range(30):
            res1 = self.text_tools_rcn.find_text(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=df_contacts, return_only_df=True)
            if res1 is not None:
                df_name = 'df_contacts'
                break

            res1 = self.text_tools_rcn.find_text(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=df_101, return_only_df=True)
            if res1 is not None:
                df_name = 'df_101'
                break

            res1 = self.text_tools_rcn.find_text(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=df_C, return_only_df=True)
            if res1 is not None:
                df_name =  'df_C'
                break

            if side == 'left':
                dx_left += 5
            else:
                dx_right += 5

        if res1 is not None:
            return res1, df_name
        return None, None

    def _find_link_by_sh(self, df_poisk, x, y, dx_left=1, dx_right=1, dy_top=1, dy_bottom=1, side='left'):
        res1 = None
        for i in range(30):
            res1 = self.text_tools_rcn.find_cirle(x, y, dx_left, dx_right, dy_top, dy_bottom, df_poisk=df_poisk)
            if res1 is not None:
                break

            if side == 'left':
                dx_left += 5
            else:
                dx_right += 5

        if res1 is not None:
            return res1
        return None

    def _direction_contact(self, df):
        if len(df) >1:
            c1 = df.iloc[0]['text']
            c2 = df.iloc[1]['text']
            x1 = df.iloc[0]['x']
            x2 = df.iloc[1]['x']
            y1 = df.iloc[0]['y']
            y2 = df.iloc[1]['y']

            dx = abs(x1 - x2)
            dy = abs(y1 - y2)

            if dx > 2:
                if x1 < x2:
                    return {c1: ['horizontal', 'left'], c2: ['horizontal', 'right']}
                else:
                    return {c1: ['horizontal', 'right'], c2: ['horizontal', 'left']}
            if dy > 2:
                if y1 > y2:
                    return {c1: ['vertical', 'top'], c2: ['vertical', 'bottom']}
                else:
                    return {c1: ['vertical', 'bottom'], c2: ['vertical', 'top']}

    def get_loc(self, loc):
        ls = loc.split('-')
        st = ls[0]
        pr = ls[1]
        if len(pr) == 2:
            p = pr[0]
            r = pr[1]
        else:
            p = pr[0:2]
            r = pr[2]
        return st, p, r

    def set_val_df(self, df, ref, col, value):
        if df is not None:
            ser = df[df['ref'] == ref][col]
            if len(ser) > 0:
                df.loc[df['ref'] == ref, col] = value
        return df

    def get_val_df(self, df, ref, col):
        if df is not None:
            df1 = df[df['ref'] == ref]
            if len(df1) > 0:
                return df1[col].values[0]

    def get_val_df_first(self, df, col):
        if df is not None:
            df1 = df.head(1)
            if len(df1) > 0:
                return df[col].values[0]

In [ ]:
ts = Tools(text_tools_st)

ts.sx_find_st_v()

ts.init_st_v()


ts.search_sx_sh_circle()
ts.init_shina_stativ()
ts.set_pit_contact()


In [212]:
df1.head(1)['ref'].values[0]

1073761402

In [180]:
df_ls_sh_p

,st,pit,p
0,1,М,[3]
1,1,МСХ,[6]
2,1,П,[4]
3,1,СХМ,[5]
4,2,П,"[2, 4, 6, 8]"
5,3,П,"[2, 4, 6, 8]"


In [ ]:

# сгруппировать по sh_st и добавить в новый столбец range для каждой группы
df_shina['contact'] = df_shina.groupby('sh_st').cumcount()
df_shina['contact'] += 1
df_shina

In [31]:
df_shina = None

df_sh = text_tools_rcn.df_match(r'[1-3]-Ш')

for i in range(len(df_sh)):
    ref1 = df_sh.iloc[i]['ref']
    name_sh = df_sh.iloc[i]['text'] # '2-Ш6'
    number_sh1 = name_sh[0] # '2'
    number_sh2 = name_sh.split('Ш')[1] # '6'
    name_sh1 = ds['text'] # 'П'
    name_sh_st = f'{number_sh2}Ш-{name_sh1}' # '4Ш-П'
    ds = text_tools_rcn.find_text_ref(ref1, 5, 5, 0, 5)
    ds1 = text_tools_rcn.find_line(ds['x'], ds['y'], 5, 5, 0, 10)
    df1 = ds1['df']
    is_line = False
    for i in range(len(df1)):
        res = df1.iloc[i]
        if res['x1'] == res['x2']:
            is_line = True
            break
    if is_line:
        df2 = text_tools_rcn.find_cirle_ref(ds['ref'], 5, 5, 0, 180, radius_lt=2)['df']
        # df2 = df2[['x', 'y', 'ref']]
        df2['sh_sx'] = name_sh
        df2['sh_st'] = name_sh_st
        df2['st'] = number_sh1
        df2['pol'] = number_sh2
        df2['pit'] = name_sh1
        if df_shina is None:
            df_shina = df2
        else:
            df_shina = df_shina.append(df2)

# сгруппировать по sh_st и добавить в новый столбец range для каждой группы
df_shina['contact'] = df_shina.groupby('sh_st').cumcount()
df_shina['contact'] += 1
df_shina

KeyError: 'text'

In [119]:
df1 = df_contacts[df_contacts['b1'].isnull()]
for i in range(len(df1)):
    res = df1.iloc[i]
df1

,loc,name,contact,ref,x,y,group,direction,side,b1,b2,l1,l2,p
132,1-82,5А,11,1073861413,7534.606597,121.286926,1-82;11:12,horizontal,left,None,None,None,None,None
133,1-82,5А,12,1073861414,7542.248568,121.286926,1-82;11:12,horizontal,right,None,None,None,None,None
138,2-68,І ГП,11,1073861835,7971.454748,129.689774,2-68;11:12,horizontal,left,None,None,None,None,None
140,2-67,ІІ ГП,11,1073861839,7992.454945,119.689563,2-67;11:12,horizontal,left,None,None,None,None,None
142,2-66,1 СП,11,1073861834,7971.454748,109.689704,2-66;11:12,horizontal,left,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,2-81,РЕЗЕРВ 1,11,1073864277,8832.455593,59.689705,2-81;11:12,horizontal,left,None,None,None,None,None
294,2-83,РЕЗЕРВ 3,11,1073864275,8832.455593,79.689775,2-83;11:12,horizontal,left,None,None,None,None,None
296,2-85,РЕЗЕРВ 5,11,1073864273,8832.455593,99.689493,2-85;11:12,horizontal,left,None,None,None,None,None
298,2-87,РЕЗЕРВ 7,11,1073864271,8832.455593,119.689562,2-87;11:12,horizontal,left,None,None,None,None,None


In [29]:
def montag_sh():
    df1 = df_contacts[df_contacts['b1'].isnull()]
    for i in range(len(df1)):
        res = df1.iloc[i]
        df2 = find_link_by_sh(df_shina, res['x'], res['y'], 10, 0, 5, 5, side=res['side'])
        if df2 is not None:
            df3 = df2['df']
            loc = res['loc']
            st, p, r = get_loc(loc)
            contact = res['contact']
            pit = df3['pit'].values[0]
            pol_sh = df3['pol'].values[0]
            contact_sh = df3['contact'].values[0]
            sh_st = df3['sh_st'].values[0]
            s1 = f'{pol_sh}Ш-{contact_sh}'
            s2 = f'{p}{r}-{contact}'
            if p == pol_sh:
                s1 = f'Ш-{contact_sh}'
                s2 = f'{r}-{contact}'

            add_st_contact(loc=loc, contact=contact, loc_to=[s1, pit])
            add_st_sh(st=st, sh_st=sh_st, contact=contact_sh, loc_to=s2)

In [164]:
df2

,x,y,radius,ref,sh_sx,sh_st,st,pol,pit
1839,8784.811661,130.729916,0.749800,1073855257,2-Ш8,8Ш-П,2,8,П
1840,8784.811661,120.729831,0.750025,1073855258,2-Ш8,8Ш-П,2,8,П
1841,8784.811661,110.729973,0.750025,1073855260,2-Ш8,8Ш-П,2,8,П
1842,8784.811661,100.729761,0.750025,1073855262,2-Ш8,8Ш-П,2,8,П
1990,8784.811661,90.729903,0.750025,1073855762,2-Ш8,8Ш-П,2,8,П
1991,8784.811661,80.729691,0.750025,1073855763,2-Ш8,8Ш-П,2,8,П
1992,8784.811661,70.729833,0.750025,1073855765,2-Ш8,8Ш-П,2,8,П
1993,8784.811661,60.729622,0.750025,1073855767,2-Ш8,8Ш-П,2,8,П
1994,8784.811661,50.729763,0.750025,1073855769,2-Ш8,8Ш-П,2,8,П
1995,8784.811661,40.729777,0.749800,1073855771,2-Ш8,8Ш-П,2,8,П


In [30]:
montag_sh()

NameError: name 'df_shina' is not defined

In [137]:
df3

,x,y,radius,ref,sh_sx,sh_st,st,pol,pit,contact
1599,7944.810841,130.729916,0.7498,1073854321,2-Ш6,6Ш-П,2,6,П,1


In [138]:
df3['sh_st'].values[0]

'6Ш-П'

In [ ]:
add_st_contact(loc=loc, contact=contact, loc_to=[f'{b}-{n}'])

In [163]:
add_st_sh(st=st, sh_st=sh_st, contact=contact_sh, loc_to='1-1')

True

In [153]:
st, sh_st, contact_sh

('2', '6Ш-П', 1)

In [148]:
df1 = text_tools.df_match(sh_st)
ref1 = df1.iloc[0]['ref']
d1 = text_tools.find_text_ref(ref1, 90, 110, -25, 40)
df1 = d1['df']
df1 = df1[df1['text'] == str(contact_sh)]
ds = df1.iloc[0].to_dict()
ds

{'text': '1',
 'x': 5367.637110027854,
 'y': 12.903365964751856,
 'ref': 1073835912}

In [149]:
str('f')

'f'

In [76]:
res = df_B.iloc[0]
res

text            2-В
x       1701.805316
y        144.729877
ref      1073748898
Name: 7069, dtype: object

In [70]:
df1, df_name1, df2, df_name2 = f1()

In [71]:
df1

,loc,name,contact,ref,x,y,group,direction,side,b1,b2,l1,l2,p
0,2-68,І ГП,82,1073741891,1673.142768,142.617286,2-68;82:22,vertical,top,None,None,None,None,None


In [72]:
df2

,text,x,y,ref
73,101-112-1,1757.055364,144.729845,1073741902


In [58]:
df_name1, df_name2

('df_contacts', 'df_101')

In [82]:
loc = df1['loc'].iloc[0]
st, pol, rd = get_loc(loc)
st, pol, rd

('2', '6', '8')

In [88]:

b_name = ds_st_b['df_101']
b_name

['В110', 'В111', 'В112', 'В210', 'В211', 'В212', 'В29']

In [110]:

df_t = df_st_b[(df_st_b['st']==st) & (df_st_b['b'].isin(b_name)) & (df_st_b['l1']=='')]
ls = ds_st_b['df_101']
df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))
df_t

,st,b,n,l1,l2,p
520,2,В110,1,,,
521,2,В110,2,,,
522,2,В110,3,,,
523,2,В110,4,,,
524,2,В110,5,,,
...,...,...,...,...,...,...
515,2,В29,16,,,
516,2,В29,17,,,
517,2,В29,18,,,
518,2,В29,19,,,


In [111]:
df_t.iloc[0]['b'],  df_t.iloc[0]['n']

('В110', 1)

In [109]:
# отсортировать по полю b в порядке как в списке ['В110', 'В111', 'В112', 'В210', 'В211', 'В212', 'В29'] и по полю n в порядке возрастания
ls = ds_st_b['df_101']
df_t = df_t.sort_values(by=['b', 'n'], key=lambda x: x.map(dict(zip(ls, range(len(ls))))))

df_t

,st,b,n,l1,l2,p
520,2,В110,1,,,
521,2,В110,2,,,
522,2,В110,3,,,
523,2,В110,4,,,
524,2,В110,5,,,
...,...,...,...,...,...,...
515,2,В29,16,,,
516,2,В29,17,,,
517,2,В29,18,,,
518,2,В29,19,,,


In [12]:
text_tools_st = {
    '1': text_tools_st1,
}

In [39]:
find_st_contact(text_tools_st, loc='1-14', contact='41', loc_to=['1', '141-42', 'СХ'])
find_st_contact(text_tools_st, loc=loc, contact=contact, loc_to=[f'{b}-{n}'])
find_st_v(text_tools_st, st, b, str(n), [loc, b_st])

In [16]:
tt = text_tools_st['c2']

In [19]:
df_v = tt.df_match('В110')
df_v

,text,x,y,ref
319,В110,8108.149716,261.669801,1073760099


In [20]:
ref_v = df_v.iloc[0]['ref']
x = df_v.iloc[0]['x']
y = df_v.iloc[0]['y']

In [21]:
ref_v, x, y

(1073760099, 8108.1497163294425, 261.669801172547)

In [31]:
d1 = tt.find_text_ref(ref_v, 5, 5, 0, 85)
df1 = d1['df']
df1

,text,x,y,ref
426,1,8103.158772,255.882376,1073760292
427,2,8109.424424,255.814233,1073760293
428,3,8103.324137,247.841570,1073760294
429,4,8109.424424,247.841570,1073760295
430,5,8103.324137,239.937050,1073760296
431,6,8109.424424,239.868908,1073760297
432,7,8103.491549,231.896245,1073760298
433,8,8109.424424,231.828103,1073760299
434,9,8103.492286,223.871223,1073760300
435,10,8109.545162,223.871223,1073760301


In [33]:
df1 = df1[df1['text'] == str(n)]
ds = df1.iloc[0].to_dict()
ds

{'text': '1',
 'x': 8103.158772357989,
 'y': 255.88237552294618,
 'ref': 1073760292}

In [32]:
df1[df1['text'] == str(n)]

,text,x,y,ref
426,1,8103.158772,255.882376,1073760292


In [30]:
df1

,text,x,y,ref


In [31]:
find_st_contact(text_tools_st, loc='1-14', contact='41', loc_to=['1', '141-42', 'СХ'])

In [15]:
find_st_contact(text_tools_st, loc=loc, contact=contact, loc_to=[f'{b}-{n}'])

In [ ]:
import threading
import queue
import time

def foo(bar, q):
    print ('hello {}'.format(bar))
    q.put('hello {}'.format(bar))
    time.sleep(5)
    return 'foo'

qe = queue.Queue()
t = threading.Thread(target=foo, args=('world!', qe))
t.start()
while t.is_alive(): # пока функция выполняется
    n = qe.get()

    print(n)
    print(type(n))

# ret = qe.join()
# print (ret)

hello world!
hello world!
<class 'str'>


In [1]:
import threading, time, queue, random

def worker(data, result):
    # цикл, пока в очереди есть задания
    while not data.empty():
        # получаем задание из очереди с данными
        task = data.get()
        # для приличия, умножим хотя бы на 2
        res = task * 2
        # результаты будем возвращать как кортеж,
        # в котором будет (результат и ID_потока)
        result.put((res, threading.get_ident()))
        # имитируем нагрузку
        t_sleep = random.uniform(0.5, 2)
        time.sleep(t_sleep)
        # говорим очереди с данными 'data',
        # что задание выполнено
        data.task_done()

# заполняем очередь заданиями для потоков
# пускай это будет простой список чисел,
# которые потоки будут возвращать
data = queue.Queue()
for i in range(10, 20):
    data.put(i)

# очередь с возвращаемыми
# результатами работы потоков
result = queue.Queue()

# создаем и запускаем потоки
for _ in range(3):
    # имена потоков будут одинаковыми, что бы можно
    # было их отличить от основного потока программы
    thread = threading.Thread(name='worker',
                              target=worker,
                              args=(data, result,))
    thread.start()

# получаем результаты работы потоков в реальном
# времени в основном потоке программы.
t_start = time.time()
# цикл, пока жив хоть один поток 'worker'
while any(th.is_alive()
          for th in threading.enumerate()
          if th.name == 'worker'):

    # !Внимание! очередь с результатами при
    # работе потоков с разной нагрузкой,
    # на короткие промежутки может быть пустой,
    # к тому же мы сразу извлекаем результаты
    if not result.empty():
        res, id_thread = result.get()
        # прошедшее время с момента запуска потоков
        tm = round(time.time() - t_start, 2)
        print(f'ThID-{id_thread}: результат {res}, время: {tm}')

ThID-8844: результат 20, время: 0.0
ThID-6560: результат 22, время: 0.0
ThID-13292: результат 24, время: 0.0
ThID-13292: результат 26, время: 1.07
ThID-6560: результат 28, время: 1.15
ThID-8844: результат 30, время: 1.34
ThID-13292: результат 32, время: 1.81
ThID-6560: результат 34, время: 2.52
ThID-13292: результат 36, время: 2.7
ThID-8844: результат 38, время: 3.34


In [ ]:
# OfficeEvents - test/demonstrate events with Word and Excel.
from win32com.client import DispatchWithEvents, Dispatch
import msvcrt, pythoncom
import time, sys
import types

import threading
stopEvent = threading.Event()

def TestExcel():
    class ExcelEvents:
        def OnNewWorkbook(self, wb):
            if type(wb) != types.InstanceType:
                raise RuntimeError, "The transformer doesnt appear to have translated this for us!"
            self.seen_events["OnNewWorkbook"] = None
        def OnWindowActivate(self, wb, wn):
            if type(wb) != types.InstanceType or type(wn) != types.InstanceType:
                raise RuntimeError, "The transformer doesnt appear to have translated this for us!"
            self.seen_events["OnWindowActivate"] = None
        def OnWindowDeactivate(self, wb, wn):
            self.seen_events["OnWindowDeactivate"] = None
        def OnSheetDeactivate(self, sh):
            self.seen_events["OnSheetDeactivate"] = None
        def OnSheetBeforeDoubleClick(self, Sh, Target, Cancel):
            if Target.Column % 2 == 0:
                print "You can double-click there..."
            else:
                print "You can not double-click there..."
            # This function is a void, so the result ends up in
            # the only ByRef - Cancel.
                return 1

    class WorkbookEvents:
        def OnActivate(self):
            print "workbook OnActivate"
        def OnBeforeRightClick(self, Target, Cancel):
            print "It's a Worksheet Event"

    e = DispatchWithEvents("Excel.Application", ExcelEvents)
    e.seen_events = {}
    e.Visible=1
    book = e.Workbooks.Add()
    book = DispatchWithEvents(book, WorkbookEvents)
    print "Have book", book
#    sheet = e.Worksheets(1)
#    sheet = DispatchWithEvents(sheet, WorksheetEvents)

    print "Double-click in a few of the Excel cells..."
    print "Press any key when finished with Excel, or wait 10 seconds..."
    if not _WaitForFinish(e, 10):
        e.Quit()
    if not _CheckSeenEvents(e, ["OnNewWorkbook", "OnWindowActivate"]):
        sys.exit(1)

def TestWord():
    class WordEvents:
        def OnDocumentChange(self):
            self.seen_events["OnDocumentChange"] = None
        def OnWindowActivate(self, doc, wn):
            self.seen_events["OnWindowActivate"] = None
        def OnQuit(self):
            self.seen_events["OnQuit"] = None
            stopEvent.set()

    w = DispatchWithEvents("Word.Application", WordEvents)
    w.seen_events = {}
    w.Visible = 1
    w.Documents.Add()
    print "Press any key when finished with Word, or wait 10 seconds..."
    if not _WaitForFinish(w, 10):
        w.Quit()
    if not _CheckSeenEvents(w, ["OnDocumentChange", "OnWindowActivate"]):
        sys.exit(1)

def _WaitForFinish(ob, timeout):
    end = time.time() + timeout
    while 1:
        if msvcrt.kbhit():
            msvcrt.getch()
            break
        pythoncom.PumpWaitingMessages()
        stopEvent.wait(.2)
        if stopEvent.isSet():
            stopEvent.clear()
            break
        try:
            if not ob.Visible:
                # Gone invisible - we need to pretend we timed
                # out, so the app is quit.
                return 0
        except pythoncom.com_error:
            # Excel is busy (eg, editing the cell) - ignore
            pass
        if time.time() > end:
            return 0
    return 1

def _CheckSeenEvents(o, events):
    rc = 1
    for e in events:
        if not o.seen_events.has_key(e):
            print "ERROR: Expected event did not trigger", e
            rc = 0
    return rc

def test():
    import sys
    if "noword" not in sys.argv[1:]:
        TestWord()
    if "noexcel" not in sys.argv[1:]:
        TestExcel()
    print "Word and Excel event tests passed."

if __name__=='__main__':
    test()